# The problem 
is about, finding most popular movies regarding Rating data. On the other hand, we need movie-name in rating. Instead of joining, we will broadcast movienames to all executers.

So what's the definition of **Popular movie regarding rating data ?**

#### steps
* load movie data and convert into dictionary then broadcast
* load rating data, use every lines into 1 to define **popular movies**

  

In [1]:
sc.version

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
1,application_1555140556054_0002,pyspark,idle,Link,Link,✔


SparkSession available as 'spark'.
u'2.4.0'

### Movie Master Data
movie-id|movie-name|movie-date|imdb-url|0|0|0|1|1|1|0|0|0|0|0|0|0|0|0|0|0|0|0

In [3]:
movieLines = sc.textFile("s3://spark-dataset-sandbox/datasets/ml-100k/u.item")

VBox()

In [4]:
movieLines.take(2)

VBox()

[u'1|Toy Story (1995)|01-Jan-1995||http://us.imdb.com/M/title-exact?Toy%20Story%20(1995)|0|0|0|1|1|1|0|0|0|0|0|0|0|0|0|0|0|0|0', u'2|GoldenEye (1995)|01-Jan-1995||http://us.imdb.com/M/title-exact?GoldenEye%20(1995)|0|1|1|0|0|0|0|0|0|0|0|0|0|0|0|0|1|0|0']

In [108]:
movies = movieLines.map(lambda lines: lines.split("|")) \
                   .map(lambda x: (int(x[0]), x[1])) \
                   .collectAsMap()

VBox()

In [109]:
movies[1]

VBox()

u'Toy Story (1995)'

In [110]:
moviesDictionary = sc.broadcast(movies)

VBox()

In [113]:
moviesDictionary.value[1]

VBox()

u'Toy Story (1995)'

### Rating Master Data
user-id,movie-id,rating,timespamp

In [114]:
ratingLines = sc.textFile("s3://spark-dataset-sandbox/datasets/ml-100k/u.data")

VBox()

In [115]:
ratingLines.take(2)

VBox()

[u'196\t242\t3\t881250949', u'186\t302\t3\t891717742']

In [157]:
ratings = ratingLines.map(lambda lines: lines.split()) \
                     .map(lambda x: (int(x[1]), 1)) \
                     .cache() \
                     .reduceByKey(lambda x, y: (x+y)) \
                     .cache() \
                     .map(lambda x: (x[1], x[0])) \
                     .sortByKey() \
                     .cache() \
                     .max() 
 

VBox()

In [159]:
moviesDictionary.value[ratings[1]], ratings[0]

VBox()

(u'Star Wars (1977)', 583)